This is the initial notebook to explore the data and run the first models. 

In [1]:
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder

from fastai.imports import *
from fastai.tabular.all import *

from numpy import random
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn import metrics
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import StratifiedKFold

import eli5
from eli5.sklearn import PermutationImportance

from scipy.stats import skew, norm
from catboost import CatBoostRegressor
import optuna
from optuna.samplers import TPESampler

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
test = pd.read_csv('test.csv')
sample = pd.read_csv('sample_submission.csv')
train = pd.read_csv('train.csv')

Based on initial analysis the train and test datasets have similar characteristics, so it will be easier to combine them for imputation and data analysis work. 

In [3]:
train_test = pd.concat([train, test], ignore_index=True)

In [4]:
pd.isnull(train_test).sum()[pd.isnull(train_test).sum() > 0]

MSZoning           4
LotFrontage      486
Alley           2721
Utilities          2
Exterior1st        1
Exterior2nd        1
MasVnrType        24
MasVnrArea        23
BsmtQual          81
BsmtCond          82
BsmtExposure      82
BsmtFinType1      79
BsmtFinSF1         1
BsmtFinType2      80
BsmtFinSF2         1
BsmtUnfSF          1
TotalBsmtSF        1
Electrical         1
BsmtFullBath       2
BsmtHalfBath       2
KitchenQual        1
Functional         2
FireplaceQu     1420
GarageType       157
GarageYrBlt      159
GarageFinish     159
GarageCars         1
GarageArea         1
GarageQual       159
GarageCond       159
PoolQC          2909
Fence           2348
MiscFeature     2814
SaleType           1
SalePrice       1459
dtype: int64

It looks like Alley, FireplaceQu, PoolQC, Fence and MiscFeature have significant numbers of missing data. So it will be best to eliminate those rows. There are a number of rows that have less than 5 rows with missing data. Since some of these are categorical and some are continuous data, their missing data will be replaced with the most frequent value. 

In [5]:
drop_high_nan=['Alley','FireplaceQu','PoolQC','Fence','MiscFeature']
train_test=train_test.drop(drop_high_nan,axis=1)
small_nan_cols = ['MSZoning', 'Utilities', 'Exterior1st', 'Exterior2nd', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 
                  'TotalBsmtSF', 'Electrical', 'BsmtFullBath', 'BsmtHalfBath', 'KitchenQual', 'Functional', 'GarageCars', 
                  'GarageArea','SaleType', 'SaleCondition']
small_impute = SimpleImputer(strategy='most_frequent')
train_test[small_nan_cols] = pd.DataFrame(small_impute.fit_transform(train_test[small_nan_cols]),columns=small_nan_cols)

The following columns seem to have one value significantly larger than the rest, and it would probably be best to use the mode, or most common, value to feel each NaN value: MasVnrType, MasVnrArea, BsmtCond, BsmtExposure, BsmtFinType2, GarageType, GarageFinish, GarageQual, and GarageCond. That is represents 9 out of the 13 columns. 

BsmtQual has two values larger than the rest: Gd and TA. But it only has 2.8% NaNs, so simply using the mode might be good enough.  

GarageYrBlt has 59 NaNs out 2919 rows which is only 2%. It has a dispersed set of values, so it might be easiest just to have any NaNs have the same value as YearBuilt. 

BsmtFinType1 has only 2.7% value of NaNs, and most two of its largest values are GLQ and Unf. It might be easiest to use the mode here. 

LotFrontage has 486 NaNs out of 2919 rows which is a pretty high 16.7%. It has a dispersed range of values, but looking at its characteristics from the describe function above, it seems to have a pretty even distribution with a mean of 10,168 and a median of 9,453. So using the mean to fill in the NaNs seems reasonable. If it turns out this value has a high impact on the predictability of the SalePrice, then it might be good to revisit this assumption. 

In [6]:
mode_cols = ['MasVnrType', 'MasVnrArea', 'BsmtCond', 'BsmtExposure', 'BsmtFinType2', 'GarageType', 'GarageFinish', 
             'GarageQual','GarageCond', 'BsmtQual', 'BsmtFinType1']
mode_impute = SimpleImputer(strategy='most_frequent')
train_test[mode_cols] = pd.DataFrame(mode_impute.fit_transform(train_test[mode_cols]),columns=mode_cols)
train_test['LotFrontage'].fillna((train_test['LotFrontage'].mean()), inplace=True)
train_test['GarageYrBlt'] = train_test['GarageYrBlt'].fillna(train_test['YearBuilt'])

In [7]:
pd.isnull(train_test).sum()[pd.isnull(train_test).sum() > 0]

SalePrice    1459
dtype: int64

In [8]:
train_test['BsmtQual'].unique()

array(['Gd', 'TA', 'Ex', 'Fa'], dtype=object)

In [9]:
train_test['BsmtFinType1'].unique()

array(['GLQ', 'ALQ', 'Unf', 'Rec', 'BLQ', 'LwQ'], dtype=object)

In [10]:
train_test.BsmtQual = train_test.BsmtQual.replace({"Ex": 110, "Gd": 95, "TA": 85, "Fa": 75, "Po": 60, "NA": 0})

In [11]:
train_test.BsmtFinType1 = train_test.BsmtFinType1.replace({"GLQ": 6, "ALQ": 5, "BLQ": 4, "Rec": 3, "LwQ": 2, "Unf": 1,
                                                         "NA": 0})

It will be necessary to identify all the columns that have non-numeric object values and then convert them to numeric values. 

In [12]:
obj_cols = list(train_test.select_dtypes(['object']).columns)
obj_cols

['MSZoning',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'MasVnrArea',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinSF1',
 'BsmtFinType2',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'BsmtFullBath',
 'BsmtHalfBath',
 'KitchenQual',
 'Functional',
 'GarageType',
 'GarageFinish',
 'GarageCars',
 'GarageArea',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'SaleType',
 'SaleCondition']

In [13]:
for column in obj_cols:
     train_test[column] = pd.factorize(train_test[column], sort=True)[0]

To create some new columns that might compound the effects of the existing columns with higher impact on the xgb score. 

In [14]:
train_test['QualCondSum'] = train_test['OverallQual'] + train_test['OverallCond']
train_test['RemodTime'] = train_test['YearRemodAdd'] - train_test['YearBuilt']
train_test['BsmtFinTypeSF1'] = train_test['BsmtFinType1'] * train_test['BsmtFinSF1']
train_test['TotalFlrSF'] = train_test['1stFlrSF'] + train_test['2ndFlrSF']
train_test['TotalFinSF'] = train_test['GrLivArea'] + train_test['BsmtFinSF1']
train_test['GarageCarArea'] = train_test['GarageArea'] * train_test['GarageCars']
train_test['TotalSF'] = train_test['1stFlrSF'] + train_test['2ndFlrSF'] + train_test['TotalBsmtSF']

To now eliminate the columns that have negative or zero influence on the xgb score. 

In [15]:
train_test.drop(['Utilities','Street','TotRmsAbvGrd','3SsnPorch','YrSold','Exterior2nd','LotConfig',
                              'HouseStyle','EnclosedPorch','WoodDeckSF','Foundation','RoofMatl','Electrical',
                              'GarageType','LotFrontage','SaleType','MoSold','BsmtExposure','1stFlrSF',
                              'BsmtFinSF1','Exterior1st','KitchenQual','TotalFlrSF'], axis=1, inplace=True)

To look at how many features have a skew above 0.5, since high skew can be an issue in regression analysis. 

In [16]:
skew_features = train_test.apply(lambda x: skew(x)).sort_values(ascending=False)

In [17]:
high_skew = skew_features[skew_features > 0.5]
skew_index = high_skew.index

In [18]:
skew_index

Index(['MiscVal', 'PoolArea', 'LotArea', 'LowQualFinSF', 'Heating',
       'Condition2', 'LandSlope', 'KitchenAbvGr', 'ScreenPorch',
       'BsmtHalfBath', 'BsmtFinSF2', 'Condition1', 'OpenPorchSF', 'BldgType',
       'RemodTime', 'MasVnrArea', 'RoofStyle', 'MSSubClass', 'GarageCarArea',
       'GrLivArea', 'TotalFinSF', 'BsmtFinTypeSF1', 'TotalSF', '2ndFlrSF',
       'BsmtQual', 'Fireplaces', 'HalfBath', 'BsmtFullBath', 'OverallCond'],
      dtype='object')

In [19]:
print("There are {} numerical features with Skew > 0.5 :".format(high_skew.shape[0]))
skewness = pd.DataFrame({'Skew' :high_skew})
skew_features.head(30)

There are 29 numerical features with Skew > 0.5 :


MiscVal           21.947195
PoolArea          16.898328
LotArea           12.822431
LowQualFinSF      12.088761
Heating           12.078788
Condition2        12.060093
LandSlope          4.975157
KitchenAbvGr       4.302254
ScreenPorch        3.946694
BsmtHalfBath       3.931594
BsmtFinSF2         3.476562
Condition1         2.983114
OpenPorchSF        2.535114
BldgType           2.192261
RemodTime          2.063712
MasVnrArea         1.556310
RoofStyle          1.553307
MSSubClass         1.375457
GarageCarArea      1.296672
GrLivArea          1.269358
TotalFinSF         1.159630
BsmtFinTypeSF1     0.932490
TotalSF            0.922771
2ndFlrSF           0.861675
BsmtQual           0.796302
Fireplaces         0.733495
HalfBath           0.694566
BsmtFullBath       0.624832
OverallCond        0.570312
HeatingQC          0.486656
dtype: float64

To normalize the features with skew above 0.5.

In [20]:
for i in skew_index:
    train_test[i] = np.log1p(train_test[i])

C:\Users\tompe\anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


To create a column with the log of the SalePrice to match the evaluation process in the contest. 

In [21]:
train_test['LogSalePrice'] = train_test['SalePrice'].apply(np.log)

To separate the train_test dataset back into the train and test datasets, identify the independent and dependent columns, and create the validation split.

In [22]:
train = train_test[train_test['SalePrice'].notnull()].copy()
test = train_test[train_test['SalePrice'].isnull()].drop(['SalePrice','LogSalePrice'],axis=1)
X = train.drop(['SalePrice','LogSalePrice'],axis=1)
y = train.LogSalePrice

In [23]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [24]:
'''def catboost_objective(trial):
    param = {  
        
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "learning_rate": trial.suggest_float("learning_rate", .001, 0.1),
        "depth": trial.suggest_int("depth", 1, 12),
        "l2_leaf_reg":trial.suggest_float("l2_leaf_reg", .0001, 25.0),
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "bagging_temperature":trial.suggest_float("bagging_temperature", 0, 10),
        "model_size_reg":trial.suggest_float("model_size_reg", 0.1, 1.0, log=True),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
            }        

    
    model = CatBoostRegressor(**param)
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=0)
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    return rmse'''

In [25]:
'''study_catboost = optuna.create_study(sampler=TPESampler(), direction='minimize')
study_catboost.optimize(catboost_objective, n_trials=1000)
print('Number of finished trials:', len(study_catboost.trials))
print('Best trial:', study_catboost.best_trial.params)'''

[I 2023-06-15 14:25:42,012] A new study created in memory with name: no-name-54afe4c3-da6e-4fac-b65e-e290ba89ad30
[I 2023-06-15 14:25:44,032] Trial 0 finished with value: 0.12696314120477328 and parameters: {'colsample_bylevel': 0.05582062739495187, 'learning_rate': 0.07916394845959324, 'depth': 3, 'l2_leaf_reg': 0.9589700063681216, 'subsample': 0.8525357623798345, 'bagging_temperature': 8.368103410643174, 'model_size_reg': 0.9209099514553063, 'boosting_type': 'Ordered'}. Best is trial 0 with value: 0.12696314120477328.
[I 2023-06-15 14:25:45,538] Trial 1 finished with value: 0.13586449265732786 and parameters: {'colsample_bylevel': 0.019508114504635833, 'learning_rate': 0.06558454091170078, 'depth': 7, 'l2_leaf_reg': 9.926353604347035, 'subsample': 0.9736287014087781, 'bagging_temperature': 9.141293097691506, 'model_size_reg': 0.37781974235158305, 'boosting_type': 'Ordered'}. Best is trial 0 with value: 0.12696314120477328.
[I 2023-06-15 14:25:46,754] Trial 2 finished with value: 0.15

[I 2023-06-15 14:27:03,533] Trial 20 finished with value: 0.13475009792756254 and parameters: {'colsample_bylevel': 0.07458773529884201, 'learning_rate': 0.07392659876588242, 'depth': 9, 'l2_leaf_reg': 3.6978153327637773, 'subsample': 0.7347541654219124, 'bagging_temperature': 5.081457166726072, 'model_size_reg': 0.9855159901641836, 'boosting_type': 'Ordered'}. Best is trial 0 with value: 0.12696314120477328.
[I 2023-06-15 14:27:05,280] Trial 21 finished with value: 0.13007580445125536 and parameters: {'colsample_bylevel': 0.059838912744675246, 'learning_rate': 0.08583263472114985, 'depth': 5, 'l2_leaf_reg': 3.5388202773159803, 'subsample': 0.8848783528678854, 'bagging_temperature': 2.7127964788021894, 'model_size_reg': 0.7899913641345258, 'boosting_type': 'Plain'}. Best is trial 0 with value: 0.12696314120477328.
[I 2023-06-15 14:27:07,483] Trial 22 finished with value: 0.13321873725063804 and parameters: {'colsample_bylevel': 0.060742454157418566, 'learning_rate': 0.08917782737395967

[I 2023-06-15 14:28:19,933] Trial 40 finished with value: 0.13998590645032774 and parameters: {'colsample_bylevel': 0.024910357333262134, 'learning_rate': 0.08592970702022863, 'depth': 1, 'l2_leaf_reg': 5.2338394932292065, 'subsample': 0.6505161438599784, 'bagging_temperature': 8.58573143645042, 'model_size_reg': 0.8613004064981176, 'boosting_type': 'Plain'}. Best is trial 0 with value: 0.12696314120477328.
[I 2023-06-15 14:28:21,220] Trial 41 finished with value: 0.14244167412620018 and parameters: {'colsample_bylevel': 0.010783294971064908, 'learning_rate': 0.07010212660987165, 'depth': 3, 'l2_leaf_reg': 3.8480123143076765, 'subsample': 0.7442860772919991, 'bagging_temperature': 4.930913748456754, 'model_size_reg': 0.8728384490321842, 'boosting_type': 'Plain'}. Best is trial 0 with value: 0.12696314120477328.
[I 2023-06-15 14:28:22,470] Trial 42 finished with value: 0.13021400627524762 and parameters: {'colsample_bylevel': 0.041261815526692024, 'learning_rate': 0.08201336916728423, '

[I 2023-06-15 14:28:53,248] Trial 60 finished with value: 0.1303631635662064 and parameters: {'colsample_bylevel': 0.05308407222870424, 'learning_rate': 0.06700805385424385, 'depth': 3, 'l2_leaf_reg': 0.8891754111240953, 'subsample': 0.7465562155885814, 'bagging_temperature': 7.126939204770908, 'model_size_reg': 0.7948990938802735, 'boosting_type': 'Plain'}. Best is trial 55 with value: 0.12510926896141228.
[I 2023-06-15 14:28:54,386] Trial 61 finished with value: 0.13595700373148945 and parameters: {'colsample_bylevel': 0.03773601553381137, 'learning_rate': 0.08392031831244287, 'depth': 2, 'l2_leaf_reg': 2.306017984089096, 'subsample': 0.6556331161160145, 'bagging_temperature': 4.905638919012867, 'model_size_reg': 0.9211433580158694, 'boosting_type': 'Plain'}. Best is trial 55 with value: 0.12510926896141228.
[I 2023-06-15 14:28:55,884] Trial 62 finished with value: 0.12735243079588474 and parameters: {'colsample_bylevel': 0.043716140846883966, 'learning_rate': 0.0868471972825816, 'de

[I 2023-06-15 14:29:51,148] Trial 80 finished with value: 0.1310378763144958 and parameters: {'colsample_bylevel': 0.039666500098139275, 'learning_rate': 0.09318604932655852, 'depth': 2, 'l2_leaf_reg': 2.4920731572484707, 'subsample': 0.8246205707859691, 'bagging_temperature': 6.26152728000392, 'model_size_reg': 0.8748639647837457, 'boosting_type': 'Ordered'}. Best is trial 55 with value: 0.12510926896141228.
[I 2023-06-15 14:29:52,548] Trial 81 finished with value: 0.12899278932792196 and parameters: {'colsample_bylevel': 0.04538089601576495, 'learning_rate': 0.0970452173726886, 'depth': 2, 'l2_leaf_reg': 0.585347493859919, 'subsample': 0.7038895719180646, 'bagging_temperature': 4.686384919758386, 'model_size_reg': 0.9554405658606617, 'boosting_type': 'Plain'}. Best is trial 55 with value: 0.12510926896141228.
[I 2023-06-15 14:29:53,806] Trial 82 finished with value: 0.13583153850000076 and parameters: {'colsample_bylevel': 0.047699593669747915, 'learning_rate': 0.08082869138883618, '

...

[I 2023-06-15 14:48:01,599] Trial 720 finished with value: 0.12753605183061784 and parameters: {'colsample_bylevel': 0.05941453759514873, 'learning_rate': 0.0740498802755282, 'depth': 3, 'l2_leaf_reg': 1.781919776336462, 'subsample': 0.6286235504495673, 'bagging_temperature': 2.5970197096071934, 'model_size_reg': 0.8266466893582592, 'boosting_type': 'Plain'}. Best is trial 528 with value: 0.12324749357375206.
[I 2023-06-15 14:48:03,333] Trial 721 finished with value: 0.13184278734408328 and parameters: {'colsample_bylevel': 0.06968646540472968, 'learning_rate': 0.0768875018787629, 'depth': 2, 'l2_leaf_reg': 15.02033917813279, 'subsample': 0.22256528920406177, 'bagging_temperature': 6.774417814757927, 'model_size_reg': 0.7379259896391677, 'boosting_type': 'Plain'}. Best is trial 528 with value: 0.12324749357375206.
[I 2023-06-15 14:48:05,389] Trial 722 finished with value: 0.13073085944998492 and parameters: {'colsample_bylevel': 0.051673567800003144, 'learning_rate': 0.0584940328633938

[I 2023-06-15 14:48:33,364] Trial 740 finished with value: 0.12667307725656093 and parameters: {'colsample_bylevel': 0.03874224259141236, 'learning_rate': 0.07372996324734092, 'depth': 3, 'l2_leaf_reg': 2.9387460179165172, 'subsample': 0.7009313975471843, 'bagging_temperature': 0.15128098074410068, 'model_size_reg': 0.714927843526683, 'boosting_type': 'Plain'}. Best is trial 528 with value: 0.12324749357375206.
[I 2023-06-15 14:48:34,791] Trial 741 finished with value: 0.13083654908064735 and parameters: {'colsample_bylevel': 0.05738047382810049, 'learning_rate': 0.08415155220726792, 'depth': 2, 'l2_leaf_reg': 21.532568214312384, 'subsample': 0.2986582869758009, 'bagging_temperature': 4.731578768179878, 'model_size_reg': 0.9627047771084482, 'boosting_type': 'Plain'}. Best is trial 528 with value: 0.12324749357375206.
[I 2023-06-15 14:48:36,350] Trial 742 finished with value: 0.15333344195658005 and parameters: {'colsample_bylevel': 0.07958612551275326, 'learning_rate': 0.00843151134502

[I 2023-06-15 14:50:09,653] Trial 760 finished with value: 0.12800711117711905 and parameters: {'colsample_bylevel': 0.0442397098401892, 'learning_rate': 0.07169412642140197, 'depth': 3, 'l2_leaf_reg': 5.8792237953359345, 'subsample': 0.886264673568641, 'bagging_temperature': 7.23996856005405, 'model_size_reg': 0.36539134856836875, 'boosting_type': 'Plain'}. Best is trial 528 with value: 0.12324749357375206.
[I 2023-06-15 14:50:11,399] Trial 761 finished with value: 0.13097044164729513 and parameters: {'colsample_bylevel': 0.052435892719543506, 'learning_rate': 0.061151401503700846, 'depth': 4, 'l2_leaf_reg': 0.3678549061311618, 'subsample': 0.29782072586393094, 'bagging_temperature': 5.87950275519073, 'model_size_reg': 0.12627231743754935, 'boosting_type': 'Plain'}. Best is trial 528 with value: 0.12324749357375206.
[I 2023-06-15 14:50:12,915] Trial 762 finished with value: 0.13020270022929475 and parameters: {'colsample_bylevel': 0.05626193734479044, 'learning_rate': 0.05159474592329

[I 2023-06-15 14:50:44,122] Trial 780 finished with value: 0.12743553570644356 and parameters: {'colsample_bylevel': 0.07222376033049288, 'learning_rate': 0.09183440057252543, 'depth': 3, 'l2_leaf_reg': 3.9230090533078705, 'subsample': 0.8677168697877016, 'bagging_temperature': 7.825477629848357, 'model_size_reg': 0.7050892462559644, 'boosting_type': 'Plain'}. Best is trial 528 with value: 0.12324749357375206.
[I 2023-06-15 14:50:45,613] Trial 781 finished with value: 0.14047129019372395 and parameters: {'colsample_bylevel': 0.08258999010156964, 'learning_rate': 0.0210994833820841, 'depth': 2, 'l2_leaf_reg': 24.143265869980013, 'subsample': 0.5116946208731294, 'bagging_temperature': 9.672669911710797, 'model_size_reg': 0.15579468708816843, 'boosting_type': 'Plain'}. Best is trial 528 with value: 0.12324749357375206.
[I 2023-06-15 14:50:46,847] Trial 782 finished with value: 0.13103568032775267 and parameters: {'colsample_bylevel': 0.06882141083360468, 'learning_rate': 0.072492385869364

KeyboardInterrupt: 

In [70]:
cat_model = CatBoostRegressor(colsample_bylevel=0.08004892886595912,
                                      learning_rate=0.08724584964954875,
                                      depth=3,
                                      l2_leaf_reg=20.46786447675221,
                                      subsample=0.8203440457879617,
                                      bagging_temperature=4.50221817725758,
                                      model_size_reg=0.9722196114116113,
                                      boosting_type='Plain')

In [71]:
X,y = shuffle(X,y, random_state=42)
X = X.reset_index(drop=True)
y = y.reset_index(drop=True)

In [72]:
def rmse_cv(model,X,y):
    rmse = np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=6))
    return rmse

In [73]:
print(rmse_cv(cat_model,X,y).mean())

0:	learn: 0.3757329	total: 1.08ms	remaining: 1.08s
1:	learn: 0.3611354	total: 2.32ms	remaining: 1.16s
2:	learn: 0.3474503	total: 3.17ms	remaining: 1.05s
3:	learn: 0.3339523	total: 4.04ms	remaining: 1s
4:	learn: 0.3209996	total: 4.8ms	remaining: 955ms
5:	learn: 0.3112332	total: 5.59ms	remaining: 926ms
6:	learn: 0.2991561	total: 6.27ms	remaining: 890ms
7:	learn: 0.2893737	total: 6.94ms	remaining: 861ms
8:	learn: 0.2788741	total: 7.7ms	remaining: 848ms
9:	learn: 0.2700166	total: 8.47ms	remaining: 839ms
10:	learn: 0.2622585	total: 9.31ms	remaining: 837ms
11:	learn: 0.2542313	total: 10.4ms	remaining: 855ms
12:	learn: 0.2489426	total: 11.4ms	remaining: 868ms
13:	learn: 0.2422726	total: 12.3ms	remaining: 863ms
14:	learn: 0.2364211	total: 13ms	remaining: 856ms
15:	learn: 0.2308106	total: 14ms	remaining: 861ms
16:	learn: 0.2266639	total: 14.9ms	remaining: 860ms
17:	learn: 0.2215121	total: 15.7ms	remaining: 855ms
18:	learn: 0.2180605	total: 16.5ms	remaining: 853ms
19:	learn: 0.2124035	total: 17.

235:	learn: 0.1090778	total: 186ms	remaining: 603ms
236:	learn: 0.1090216	total: 187ms	remaining: 603ms
237:	learn: 0.1086653	total: 188ms	remaining: 603ms
238:	learn: 0.1086299	total: 189ms	remaining: 601ms
239:	learn: 0.1085729	total: 190ms	remaining: 600ms
240:	learn: 0.1084345	total: 190ms	remaining: 599ms
241:	learn: 0.1084078	total: 191ms	remaining: 598ms
242:	learn: 0.1083686	total: 192ms	remaining: 597ms
243:	learn: 0.1083503	total: 192ms	remaining: 596ms
244:	learn: 0.1082975	total: 193ms	remaining: 595ms
245:	learn: 0.1081499	total: 194ms	remaining: 594ms
246:	learn: 0.1080482	total: 194ms	remaining: 593ms
247:	learn: 0.1080015	total: 195ms	remaining: 592ms
248:	learn: 0.1078617	total: 196ms	remaining: 591ms
249:	learn: 0.1078260	total: 197ms	remaining: 590ms
250:	learn: 0.1078149	total: 197ms	remaining: 589ms
251:	learn: 0.1077936	total: 198ms	remaining: 588ms
252:	learn: 0.1076186	total: 199ms	remaining: 587ms
253:	learn: 0.1075695	total: 200ms	remaining: 587ms
254:	learn: 

466:	learn: 0.0959622	total: 365ms	remaining: 416ms
467:	learn: 0.0959346	total: 366ms	remaining: 416ms
468:	learn: 0.0958584	total: 366ms	remaining: 415ms
469:	learn: 0.0958034	total: 367ms	remaining: 414ms
470:	learn: 0.0957502	total: 368ms	remaining: 413ms
471:	learn: 0.0957418	total: 368ms	remaining: 412ms
472:	learn: 0.0957308	total: 369ms	remaining: 411ms
473:	learn: 0.0956949	total: 370ms	remaining: 410ms
474:	learn: 0.0956840	total: 370ms	remaining: 409ms
475:	learn: 0.0956722	total: 371ms	remaining: 408ms
476:	learn: 0.0956674	total: 372ms	remaining: 408ms
477:	learn: 0.0956145	total: 373ms	remaining: 407ms
478:	learn: 0.0956091	total: 373ms	remaining: 406ms
479:	learn: 0.0955895	total: 374ms	remaining: 405ms
480:	learn: 0.0955854	total: 374ms	remaining: 404ms
481:	learn: 0.0955793	total: 375ms	remaining: 403ms
482:	learn: 0.0955581	total: 376ms	remaining: 402ms
483:	learn: 0.0955484	total: 377ms	remaining: 401ms
484:	learn: 0.0954425	total: 377ms	remaining: 401ms
485:	learn: 

707:	learn: 0.0881377	total: 545ms	remaining: 225ms
708:	learn: 0.0881306	total: 546ms	remaining: 224ms
709:	learn: 0.0880952	total: 547ms	remaining: 223ms
710:	learn: 0.0880916	total: 548ms	remaining: 223ms
711:	learn: 0.0880481	total: 548ms	remaining: 222ms
712:	learn: 0.0880342	total: 549ms	remaining: 221ms
713:	learn: 0.0880176	total: 550ms	remaining: 220ms
714:	learn: 0.0879524	total: 551ms	remaining: 219ms
715:	learn: 0.0879063	total: 551ms	remaining: 219ms
716:	learn: 0.0878717	total: 552ms	remaining: 218ms
717:	learn: 0.0878613	total: 553ms	remaining: 217ms
718:	learn: 0.0878061	total: 553ms	remaining: 216ms
719:	learn: 0.0878005	total: 554ms	remaining: 215ms
720:	learn: 0.0877284	total: 555ms	remaining: 215ms
721:	learn: 0.0876812	total: 556ms	remaining: 214ms
722:	learn: 0.0876744	total: 556ms	remaining: 213ms
723:	learn: 0.0876695	total: 557ms	remaining: 212ms
724:	learn: 0.0876412	total: 558ms	remaining: 212ms
725:	learn: 0.0875659	total: 558ms	remaining: 211ms
726:	learn: 

949:	learn: 0.0819338	total: 725ms	remaining: 38.2ms
950:	learn: 0.0819163	total: 726ms	remaining: 37.4ms
951:	learn: 0.0818832	total: 727ms	remaining: 36.7ms
952:	learn: 0.0818804	total: 728ms	remaining: 35.9ms
953:	learn: 0.0818751	total: 729ms	remaining: 35.1ms
954:	learn: 0.0818588	total: 729ms	remaining: 34.4ms
955:	learn: 0.0818548	total: 730ms	remaining: 33.6ms
956:	learn: 0.0818501	total: 731ms	remaining: 32.8ms
957:	learn: 0.0818122	total: 732ms	remaining: 32.1ms
958:	learn: 0.0818064	total: 732ms	remaining: 31.3ms
959:	learn: 0.0817780	total: 733ms	remaining: 30.5ms
960:	learn: 0.0817045	total: 734ms	remaining: 29.8ms
961:	learn: 0.0816534	total: 735ms	remaining: 29ms
962:	learn: 0.0816448	total: 735ms	remaining: 28.3ms
963:	learn: 0.0816408	total: 736ms	remaining: 27.5ms
964:	learn: 0.0816369	total: 737ms	remaining: 26.7ms
965:	learn: 0.0816193	total: 737ms	remaining: 26ms
966:	learn: 0.0816019	total: 738ms	remaining: 25.2ms
967:	learn: 0.0815988	total: 739ms	remaining: 24.4

...

3:	learn: 0.3537453	total: 4.1ms	remaining: 1.02s
4:	learn: 0.3390879	total: 5.2ms	remaining: 1.03s
5:	learn: 0.3301115	total: 6.14ms	remaining: 1.02s
6:	learn: 0.3188860	total: 7.29ms	remaining: 1.03s
7:	learn: 0.3080384	total: 8.06ms	remaining: 1000ms
8:	learn: 0.2998440	total: 8.74ms	remaining: 962ms
9:	learn: 0.2886128	total: 11.4ms	remaining: 1.13s
10:	learn: 0.2780102	total: 13.4ms	remaining: 1.2s
11:	learn: 0.2696499	total: 14.3ms	remaining: 1.18s
12:	learn: 0.2628988	total: 15.1ms	remaining: 1.15s
13:	learn: 0.2556730	total: 16.2ms	remaining: 1.14s
14:	learn: 0.2515174	total: 16.9ms	remaining: 1.11s
15:	learn: 0.2460410	total: 17.7ms	remaining: 1.08s
16:	learn: 0.2421776	total: 18.4ms	remaining: 1.06s
17:	learn: 0.2362955	total: 19.1ms	remaining: 1.04s
18:	learn: 0.2329741	total: 20.1ms	remaining: 1.03s
19:	learn: 0.2278039	total: 21.4ms	remaining: 1.05s
20:	learn: 0.2236789	total: 22.9ms	remaining: 1.07s
21:	learn: 0.2194995	total: 25.2ms	remaining: 1.12s
22:	learn: 0.2143943	

202:	learn: 0.1148767	total: 181ms	remaining: 712ms
203:	learn: 0.1147920	total: 182ms	remaining: 711ms
204:	learn: 0.1147446	total: 183ms	remaining: 711ms
205:	learn: 0.1146952	total: 184ms	remaining: 709ms
206:	learn: 0.1146125	total: 185ms	remaining: 707ms
207:	learn: 0.1145695	total: 185ms	remaining: 706ms
208:	learn: 0.1143846	total: 186ms	remaining: 703ms
209:	learn: 0.1142008	total: 187ms	remaining: 702ms
210:	learn: 0.1141234	total: 187ms	remaining: 700ms
211:	learn: 0.1141064	total: 188ms	remaining: 698ms
212:	learn: 0.1141062	total: 188ms	remaining: 695ms
213:	learn: 0.1140761	total: 189ms	remaining: 693ms
214:	learn: 0.1140106	total: 189ms	remaining: 692ms
215:	learn: 0.1139479	total: 190ms	remaining: 691ms
216:	learn: 0.1138953	total: 191ms	remaining: 689ms
217:	learn: 0.1138805	total: 192ms	remaining: 687ms
218:	learn: 0.1136997	total: 192ms	remaining: 685ms
219:	learn: 0.1136587	total: 193ms	remaining: 684ms
220:	learn: 0.1136226	total: 194ms	remaining: 682ms
221:	learn: 

462:	learn: 0.0994944	total: 362ms	remaining: 420ms
463:	learn: 0.0994753	total: 363ms	remaining: 419ms
464:	learn: 0.0994244	total: 364ms	remaining: 419ms
465:	learn: 0.0993702	total: 365ms	remaining: 418ms
466:	learn: 0.0993498	total: 365ms	remaining: 417ms
467:	learn: 0.0992947	total: 366ms	remaining: 416ms
468:	learn: 0.0992288	total: 367ms	remaining: 415ms
469:	learn: 0.0991976	total: 367ms	remaining: 414ms
470:	learn: 0.0991335	total: 368ms	remaining: 414ms
471:	learn: 0.0991253	total: 369ms	remaining: 413ms
472:	learn: 0.0989785	total: 370ms	remaining: 412ms
473:	learn: 0.0989552	total: 370ms	remaining: 411ms
474:	learn: 0.0989269	total: 371ms	remaining: 410ms
475:	learn: 0.0989135	total: 371ms	remaining: 409ms
476:	learn: 0.0988009	total: 372ms	remaining: 408ms
477:	learn: 0.0987580	total: 373ms	remaining: 407ms
478:	learn: 0.0986678	total: 373ms	remaining: 406ms
479:	learn: 0.0986423	total: 374ms	remaining: 405ms
480:	learn: 0.0986259	total: 374ms	remaining: 404ms
481:	learn: 

717:	learn: 0.0901264	total: 542ms	remaining: 213ms
718:	learn: 0.0900943	total: 543ms	remaining: 212ms
719:	learn: 0.0900873	total: 544ms	remaining: 212ms
720:	learn: 0.0900481	total: 545ms	remaining: 211ms
721:	learn: 0.0900367	total: 546ms	remaining: 210ms
722:	learn: 0.0900301	total: 546ms	remaining: 209ms
723:	learn: 0.0899794	total: 547ms	remaining: 209ms
724:	learn: 0.0899732	total: 548ms	remaining: 208ms
725:	learn: 0.0899493	total: 548ms	remaining: 207ms
726:	learn: 0.0899149	total: 549ms	remaining: 206ms
727:	learn: 0.0899120	total: 550ms	remaining: 205ms
728:	learn: 0.0899037	total: 550ms	remaining: 205ms
729:	learn: 0.0898394	total: 551ms	remaining: 204ms
730:	learn: 0.0898343	total: 552ms	remaining: 203ms
731:	learn: 0.0897885	total: 552ms	remaining: 202ms
732:	learn: 0.0897225	total: 553ms	remaining: 201ms
733:	learn: 0.0896942	total: 554ms	remaining: 201ms
734:	learn: 0.0896884	total: 554ms	remaining: 200ms
735:	learn: 0.0896613	total: 555ms	remaining: 199ms
736:	learn: 

959:	learn: 0.0835387	total: 724ms	remaining: 30.1ms
960:	learn: 0.0835243	total: 725ms	remaining: 29.4ms
961:	learn: 0.0834984	total: 725ms	remaining: 28.6ms
962:	learn: 0.0834909	total: 726ms	remaining: 27.9ms
963:	learn: 0.0834822	total: 727ms	remaining: 27.1ms
964:	learn: 0.0834793	total: 728ms	remaining: 26.4ms
965:	learn: 0.0834753	total: 728ms	remaining: 25.6ms
966:	learn: 0.0834399	total: 729ms	remaining: 24.9ms
967:	learn: 0.0834192	total: 730ms	remaining: 24.1ms
968:	learn: 0.0834147	total: 730ms	remaining: 23.4ms
969:	learn: 0.0833456	total: 731ms	remaining: 22.6ms
970:	learn: 0.0833242	total: 732ms	remaining: 21.9ms
971:	learn: 0.0832474	total: 733ms	remaining: 21.1ms
972:	learn: 0.0832373	total: 733ms	remaining: 20.3ms
973:	learn: 0.0832049	total: 734ms	remaining: 19.6ms
974:	learn: 0.0831877	total: 735ms	remaining: 18.8ms
975:	learn: 0.0831648	total: 735ms	remaining: 18.1ms
976:	learn: 0.0831289	total: 736ms	remaining: 17.3ms
977:	learn: 0.0831058	total: 737ms	remaining: 

In [74]:
submit = test[['Id']]
submit = submit.reset_index(drop=True)

In [75]:
cat_model.fit(X,y)

0:	learn: 0.3839057	total: 822us	remaining: 822ms
1:	learn: 0.3686327	total: 2.26ms	remaining: 1.13s
2:	learn: 0.3523335	total: 3.29ms	remaining: 1.09s
3:	learn: 0.3386238	total: 4.25ms	remaining: 1.06s
4:	learn: 0.3253388	total: 5.17ms	remaining: 1.03s
5:	learn: 0.3153386	total: 5.99ms	remaining: 992ms
6:	learn: 0.3029196	total: 6.74ms	remaining: 956ms
7:	learn: 0.2929963	total: 7.45ms	remaining: 924ms
8:	learn: 0.2827511	total: 8.24ms	remaining: 907ms
9:	learn: 0.2735166	total: 9.04ms	remaining: 895ms
10:	learn: 0.2654625	total: 9.91ms	remaining: 891ms
11:	learn: 0.2569053	total: 10.7ms	remaining: 877ms
12:	learn: 0.2516382	total: 11.5ms	remaining: 875ms
13:	learn: 0.2450370	total: 12.8ms	remaining: 900ms
14:	learn: 0.2389828	total: 13.8ms	remaining: 905ms
15:	learn: 0.2335393	total: 14.9ms	remaining: 916ms
16:	learn: 0.2287827	total: 16ms	remaining: 923ms
17:	learn: 0.2235564	total: 16.8ms	remaining: 918ms
18:	learn: 0.2201828	total: 17.8ms	remaining: 917ms
19:	learn: 0.2144423	tota

228:	learn: 0.1112684	total: 182ms	remaining: 612ms
229:	learn: 0.1111868	total: 183ms	remaining: 612ms
230:	learn: 0.1110638	total: 184ms	remaining: 612ms
231:	learn: 0.1109915	total: 184ms	remaining: 611ms
232:	learn: 0.1109628	total: 185ms	remaining: 610ms
233:	learn: 0.1108027	total: 186ms	remaining: 609ms
234:	learn: 0.1107482	total: 187ms	remaining: 608ms
235:	learn: 0.1106810	total: 187ms	remaining: 607ms
236:	learn: 0.1105571	total: 188ms	remaining: 606ms
237:	learn: 0.1103984	total: 189ms	remaining: 604ms
238:	learn: 0.1102696	total: 190ms	remaining: 603ms
239:	learn: 0.1101714	total: 190ms	remaining: 602ms
240:	learn: 0.1101601	total: 191ms	remaining: 601ms
241:	learn: 0.1099487	total: 192ms	remaining: 600ms
242:	learn: 0.1097987	total: 193ms	remaining: 600ms
243:	learn: 0.1097412	total: 193ms	remaining: 599ms
244:	learn: 0.1095580	total: 194ms	remaining: 598ms
245:	learn: 0.1095266	total: 195ms	remaining: 597ms
246:	learn: 0.1094817	total: 196ms	remaining: 597ms
247:	learn: 

452:	learn: 0.0985179	total: 363ms	remaining: 438ms
453:	learn: 0.0984756	total: 364ms	remaining: 437ms
454:	learn: 0.0984692	total: 365ms	remaining: 437ms
455:	learn: 0.0984597	total: 366ms	remaining: 436ms
456:	learn: 0.0984472	total: 366ms	remaining: 435ms
457:	learn: 0.0983342	total: 367ms	remaining: 434ms
458:	learn: 0.0983278	total: 368ms	remaining: 433ms
459:	learn: 0.0983102	total: 369ms	remaining: 433ms
460:	learn: 0.0983017	total: 369ms	remaining: 431ms
461:	learn: 0.0982802	total: 370ms	remaining: 431ms
462:	learn: 0.0982663	total: 371ms	remaining: 430ms
463:	learn: 0.0982438	total: 371ms	remaining: 429ms
464:	learn: 0.0982197	total: 372ms	remaining: 428ms
465:	learn: 0.0981487	total: 373ms	remaining: 427ms
466:	learn: 0.0981036	total: 374ms	remaining: 426ms
467:	learn: 0.0980698	total: 374ms	remaining: 425ms
468:	learn: 0.0980339	total: 375ms	remaining: 425ms
469:	learn: 0.0980030	total: 376ms	remaining: 424ms
470:	learn: 0.0979659	total: 377ms	remaining: 423ms
471:	learn: 

686:	learn: 0.0908868	total: 543ms	remaining: 248ms
687:	learn: 0.0908498	total: 544ms	remaining: 247ms
688:	learn: 0.0907963	total: 545ms	remaining: 246ms
689:	learn: 0.0907794	total: 546ms	remaining: 245ms
690:	learn: 0.0907773	total: 547ms	remaining: 245ms
691:	learn: 0.0907704	total: 548ms	remaining: 244ms
692:	learn: 0.0907116	total: 549ms	remaining: 243ms
693:	learn: 0.0907032	total: 549ms	remaining: 242ms
694:	learn: 0.0906705	total: 550ms	remaining: 241ms
695:	learn: 0.0906562	total: 551ms	remaining: 241ms
696:	learn: 0.0906230	total: 552ms	remaining: 240ms
697:	learn: 0.0905756	total: 553ms	remaining: 239ms
698:	learn: 0.0905714	total: 554ms	remaining: 238ms
699:	learn: 0.0905260	total: 555ms	remaining: 238ms
700:	learn: 0.0905241	total: 555ms	remaining: 237ms
701:	learn: 0.0904820	total: 556ms	remaining: 236ms
702:	learn: 0.0904735	total: 557ms	remaining: 235ms
703:	learn: 0.0904351	total: 558ms	remaining: 234ms
704:	learn: 0.0904206	total: 558ms	remaining: 234ms
705:	learn: 

915:	learn: 0.0851968	total: 726ms	remaining: 66.6ms
916:	learn: 0.0851832	total: 727ms	remaining: 65.8ms
917:	learn: 0.0851403	total: 728ms	remaining: 65ms
918:	learn: 0.0851237	total: 729ms	remaining: 64.2ms
919:	learn: 0.0851197	total: 729ms	remaining: 63.4ms
920:	learn: 0.0851197	total: 730ms	remaining: 62.6ms
921:	learn: 0.0850529	total: 730ms	remaining: 61.8ms
922:	learn: 0.0850490	total: 731ms	remaining: 61ms
923:	learn: 0.0850464	total: 732ms	remaining: 60.2ms
924:	learn: 0.0850413	total: 733ms	remaining: 59.4ms
925:	learn: 0.0850403	total: 733ms	remaining: 58.6ms
926:	learn: 0.0849862	total: 734ms	remaining: 57.8ms
927:	learn: 0.0849552	total: 735ms	remaining: 57ms
928:	learn: 0.0849328	total: 735ms	remaining: 56.2ms
929:	learn: 0.0849277	total: 736ms	remaining: 55.4ms
930:	learn: 0.0849241	total: 737ms	remaining: 54.6ms
931:	learn: 0.0848693	total: 738ms	remaining: 53.8ms
932:	learn: 0.0848539	total: 739ms	remaining: 53ms
933:	learn: 0.0848466	total: 739ms	remaining: 52.2ms
9

In [76]:
#submit_predict = cat_model.predict(test)
#submit_predict = np.exp(submit_predict)

In [77]:
submit['SalePrice'] = submit_predict

In [78]:
submit.to_csv('submit_optuna_catboost.csv', index=False)

Reducing feature skewness actually slightly increased the Kaggle score. 